In [28]:
%%time
import cv2
import os
import numpy as np
from PIL import Image
import glob
from tqdm import tqdm_notebook as tqdm

step_sec = 1
os.makedirs('frames', exist_ok=True)

for fname in tqdm(glob.glob('downloaded/*.mp4')[:1]):
    fname_w_o_extension = fname[:-4]
    video = cv2.VideoCapture(fname)
    fps = video.get(cv2.CAP_PROP_FPS)
    time_length = video.get(cv2.CAP_PROP_FRAME_COUNT)
    frame_seq = 0
    count_sec = 0
    while True:         
        frame_no = (frame_seq /(time_length*fps))
        video.set(cv2.CAP_PROP_POS_FRAMES, fps*count_sec)
        ret, frame = video.read()
        if ret:
            cv2.imwrite(f"frames/{fname_w_o_extension}_{count_sec}.jpg", frame)
            print('saved')
            frame_seq += fps
            count_sec += step_sec
        else:
            break

<timed exec>:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1 [00:00<?, ?it/s]

saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
save

In [36]:
!rm -rf frames/*

In [9]:
import cv2
import os
import numpy as np
from PIL import Image
import glob
from tqdm import tqdm_notebook as tqdm
os.makedirs('frames', exist_ok=True)

for fname in tqdm(os.listdir('videos')):
    if fname.endswith('.mp4'):
        fname_w_o_extension = fname[:-4]
        video = cv2.VideoCapture(f"videos/{fname}")
        fps = video.get(cv2.CAP_PROP_FPS)
        time_length = video.get(cv2.CAP_PROP_FRAME_COUNT)
        frame_seq = 0
        count_sec = 0
        while True:         
            frame_no = (frame_seq /(time_length*fps))
            video.set(cv2.CAP_PROP_POS_FRAMES, fps*count_sec)
            ret, frame = video.read()
            if ret:
                cv2.imwrite(f"frames/{fname_w_o_extension}_{count_sec}.jpg", frame)
                frame_seq += fps
                count_sec += 1
            else:
                break

/tmp/ipykernel_16571/183127850.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for fname in tqdm(os.listdir('videos')):


  0%|          | 0/14 [00:00<?, ?it/s]

[h264 @ 0x16174280] mmco: unref short failure
[h264 @ 0x16174280] mmco: unref short failure
[h264 @ 0x16174280] mmco: unref short failure
[h264 @ 0x16174280] mmco: unref short failure
[h264 @ 0x16174280] mmco: unref short failure
[h264 @ 0x16174280] mmco: unref short failure
[h264 @ 0x16174280] mmco: unref short failure
[h264 @ 0x16174280] mmco: unref short failure
[h264 @ 0x16174280] mmco: unref short failure
[h264 @ 0x16174280] mmco: unref short failure
[h264 @ 0x16174280] mmco: unref short failure
[h264 @ 0x16174280] mmco: unref short failure
[h264 @ 0x16174280] mmco: unref short failure
[h264 @ 0x16174280] mmco: unref short failure
[h264 @ 0x16174280] mmco: unref short failure
[h264 @ 0x16174280] mmco: unref short failure
[h264 @ 0x16174280] mmco: unref short failure
[h264 @ 0x16174280] mmco: unref short failure
[h264 @ 0x16174280] mmco: unref short failure
[h264 @ 0x16174280] mmco: unref short failure
[h264 @ 0x16174280] mmco: unref short failure
[h264 @ 0x16174280] mmco: unref sh

In [32]:
! pwd

/home/vkhack/videogen


In [16]:
fname, time_length

('downloaded/dNCWe_6HAM8.mp4', 4088.0)

In [14]:
ret

False

In [12]:
! ls

app.py				     img
clip_embedder.py		     indexer
cut_frames_and_calculate_embs.ipynb  indexer.ipynb
data				     model.py
Dockerfile			     nmslib_indexer.py
download_data.ipynb		     __pycache__
downloaded			     README.md
frames_prev			     requirements.txt
frame_vectors_prev		     videos
generate_video_cut_queries.py	     wget-log


In [10]:
! ls frames/*

ls: cannot access 'frames/*': No such file or directory


In [49]:
embedder.preprocess(Image.open(fname)).unsqueeze(0).to(device)

tensor([[[[-1.4565, -1.4565, -1.4419,  ..., -1.5149, -1.5003, -1.4857],
          [-1.4565, -1.4565, -1.4419,  ..., -1.5149, -1.5003, -1.5003],
          [-1.4565, -1.4565, -1.4419,  ..., -1.5149, -1.5149, -1.5003],
          ...,
          [-0.5368, -0.8726, -1.3397,  ..., -1.5879, -1.5879, -1.5879],
          [-0.2010, -0.2886, -0.5222,  ..., -1.7631, -1.7631, -1.7485],
          [-0.0696, -0.1718, -0.2594,  ..., -1.7339, -1.7339, -1.7339]],

         [[-1.4820, -1.4820, -1.4820,  ..., -1.5120, -1.4970, -1.4820],
          [-1.4820, -1.4820, -1.4820,  ..., -1.4970, -1.4970, -1.4970],
          [-1.4820, -1.4820, -1.4820,  ..., -1.5120, -1.5120, -1.4970],
          ...,
          [-0.8066, -1.1068, -1.4519,  ..., -1.5570, -1.5420, -1.5420],
          [-0.6265, -0.6865, -0.8066,  ..., -1.7371, -1.7371, -1.7221],
          [-0.5965, -0.6715, -0.7166,  ..., -1.7071, -1.7071, -1.7071]],

         [[-0.4279, -0.4279, -0.4564,  ..., -1.1105, -1.0821, -1.0678],
          [-0.4279, -0.4279, -

In [10]:
%%time
# save embeddings
from clip_embedder import ClipEmbedder
import os
import numpy as np
from PIL import Image
from tqdm import tqdm_notebook as tqdm
os.makedirs('frame_vectors', exist_ok=True)

embedder = ClipEmbedder()
device = 'cpu'

import torch

def get_img_emb(model, img_path):
    image = model.preprocess(Image.open(img_path)).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = model.predictor.encode_image(image)
    return image_features



for fname in tqdm(os.listdir('frames')):
    if fname.endswith('.jpg'):
        fname_w_o_extension = fname[:-4]
        image_features = get_img_emb(embedder, f"frames/{fname}")
        np.save(f"frame_vectors/{fname_w_o_extension}.npy", image_features.detach().numpy(), allow_pickle=False)

<timed exec>:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/3831 [00:00<?, ?it/s]

CPU times: user 13min 11s, sys: 2.6 s, total: 13min 13s
Wall time: 3min 30s


tensor([[ 6.8788e-01,  1.3601e-01,  6.2323e-01, -2.4127e-03, -4.3132e-01,
         -1.5285e-01, -3.6626e-01,  1.8401e-01,  8.8913e-02,  2.6763e-01,
         -7.2882e-03, -4.3849e-01, -5.6040e-02,  3.8151e-01,  4.4609e-01,
         -7.3359e-02, -3.2895e-01,  2.0347e-01,  1.6974e-02,  4.5656e-02,
          1.1916e+00,  5.1098e-02,  7.2986e-02, -5.3157e-01, -1.1590e-01,
         -2.8904e-01,  1.8439e-02, -1.6722e-01,  1.3949e-01, -3.2415e-01,
         -2.8139e-01, -5.3554e-02, -5.8771e-02, -1.3012e-02, -4.1856e-01,
         -1.2045e-01, -1.4412e-01, -1.9226e-01, -1.2537e-02, -5.1212e-01,
         -4.5816e-01, -1.9985e-02,  1.6207e-02, -5.0457e-01, -5.7138e-02,
          3.6143e+00,  7.1358e-02, -3.0541e-02,  1.0498e-01,  4.9296e-01,
         -5.1986e-02, -5.8037e-02, -8.4350e-04, -3.5290e-01, -5.9726e-02,
         -5.0250e-02, -2.8818e-01, -2.3105e-01,  3.0805e-01,  4.0284e-02,
          1.0384e+00, -3.6823e-01,  2.0477e-01,  9.4019e-02, -1.5896e-01,
         -1.2109e-02, -5.7245e-01,  5.